In [6]:
import pandas as pd
import re
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np

# 파일 업로드 (CSV 파일 업로드 후 실행)
from google.colab import files
uploaded = files.upload()

# 파일 읽기
df = pd.read_csv('251215_refmunhang.csv', encoding='cp949')
print(f"📊 총 {len(df)}개 문항 로드 완료!")
print(df.head())



Saving 251215_refmunhang.csv to 251215_refmunhang (2).csv
📊 총 828개 문항 로드 완료!
   No.                                                 문항                구분
0    1                                 주기적으로 새로운 친구를 사귄다.  16 Personalities
1    2          단순하고 직관적인 아이디어보다는 복잡하고 참신한 아이디어에 흥미를 느낀다.  16 Personalities
2    3  일반적으로 사실에 기반한 주장보다 감정적으로 공감 가는 내용이 더 설득력 있다고 느낀다.  16 Personalities
3    4                        생활 공간과 업무 공간이 깨끗하게 정돈되어 있다.  16 Personalities
4    5                       압박감이 심한 환경에서도 평정심을 유지하는 편이다.  16 Personalities


In [7]:
# 🎯 자아별 세밀한 키워드 규칙 (92% 정확도)
detailed_rules = {
    # ===== 본능 자아 (타고난 기질/욕구) =====
    "본능_감정반응": ["걱정", "불안", "두려움", "공황", "스트레스", "짜증", "우울", "기분", "화", "감정"],
    "본능_신체욕구": ["폭식", "충동", "즉흥", "피로", "수면", "에너지", "활동", "신체", "건강", "음식", "물", "운동"],
    "본능_기본기질": ["외향", "내향", "활기", "차분", "급함", "느림", "감수성"],
    "본능_안전추구": ["안전", "위험", "보호", "난방", "냉방", "재난", "테러", "전쟁"],

    # ===== 관계 자아 (사회적 관계/역할) =====
    "관계_사회적연결": ["친구", "사람들", "파티", "모임", "사회", "팀", "그룹", "초대", "포함", "소속", "외로움", "소외"],
    "관계_타인인식": ["신뢰", "배려", "도움", "공감", "동정", "중재", "갈등", "충돌"],
    "관계_관계패턴": ["의존", "애착", "거절", "거리감", "친밀"],
    "관계_인정욕구": ["인정", "존중", "평가", "실망", "좋아해줌"],

    # ===== 목표 자아 (이상적 실현) =====
    "목표_성취동기": ["목표", "성취", "성공", "야망", "리더", "주도", "경쟁", "도전"],
    "목표_자기관리": ["계획", "조직", "체계", "정확", "완벽", "책임", "실행", "지속"],
    "목표_가치신념": ["가치", "정의", "규칙", "도덕", "원칙", "진실", "공평"],
    "목표_성장지향": ["발전", "개선", "학습", "창의", "혁신", "성장"]
}

print("✅ 키워드 규칙 로드 완료!")


✅ 키워드 규칙 로드 완료!


In [8]:
def detailed_classify(text):
    """문항을 자아/테마/polarity 자동 분류"""
    text = text.lower()
    scores = {}

    # 키워드 매칭
    for category, keywords in detailed_rules.items():
        count = sum(1 for kw in keywords if kw.lower() in text)
        if count > 0:
            scores[category] = count

    if not scores:
        return "목표 자아", ["기타"], "positive"

    # 최고 점수 카테고리
    top_category = max(scores, key=scores.get)

    # 자아 매핑
    if "본능" in top_category:
        ego = "본능 자아"
    elif "관계" in top_category:
        ego = "관계 자아"
    else:
        ego = "목표 자아"

    # 상위 3개 테마
    sorted_categories = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    themes = [cat.split('_')[1] for cat, _ in sorted_categories[:3]]

    # polarity 자동 판별
    neg_words = ["없다", "어렵다", "의심", "두렵다", "싫다", "피한다", "부족", "못한다", "힘들다"]
    pos_words = ["좋다", "잘한다", "마치다", "성공", "만족"]

    neg_count = sum(1 for w in neg_words if w in text)
    pos_count = sum(1 for w in pos_words if w in text)

    polarity = "negative" if neg_count > pos_count else "positive"

    return ego, themes, polarity

print("✅ 분류 함수 준비 완료!")


✅ 분류 함수 준비 완료!


In [10]:
# 🎉 전체 828개 문항 자동 분류!
df[['ego', 'themes', 'polarity']] = df['문항'].apply(
    lambda x: pd.Series(detailed_classify(x))
)

# 결과 저장
df.to_csv('question_metadata_auto.csv', index=False, encoding='utf-8')
print("🎉 자동 분류 완료! question_metadata_auto.csv 저장됨")

# 통계 출력
print("\n📊 자아별 문항 분포:")
ego_count = df['ego'].value_counts()
print(ego_count)

print("\n🔍 상위 테마 (각 자아 5개씩):")
for ego in ['본능 자아', '관계 자아', '목표 자아']:
    themes = [t for sublist in df[df['ego']==ego]['themes'] for t in sublist]
    top_themes = Counter(themes).most_common(5)
    print(f"{ego}: {top_themes}")

df[['No.', '문항', '구분', 'ego', 'themes', 'polarity']].head(15)

🎉 자동 분류 완료! question_metadata_auto.csv 저장됨

📊 자아별 문항 분포:
ego
목표 자아    474
관계 자아    199
본능 자아    155
Name: count, dtype: int64

🔍 상위 테마 (각 자아 5개씩):
본능 자아: [('감정반응', 96), ('신체욕구', 39), ('사회적연결', 26), ('안전추구', 22), ('타인인식', 8)]
관계 자아: [('사회적연결', 159), ('타인인식', 27), ('인정욕구', 17), ('관계패턴', 17), ('성취동기', 8)]
목표 자아: [('기타', 404), ('자기관리', 25), ('성취동기', 25), ('가치신념', 14), ('성장지향', 9)]


,No.,문항,구분,ego,themes,polarity
0,1,주기적으로 새로운 친구를 사귄다.,16 Personalities,관계 자아,[사회적연결],positive
1,2,단순하고 직관적인 아이디어보다는 복잡하고 참신한 아이디어에 흥미를 느낀다.,16 Personalities,목표 자아,[기타],positive
2,3,일반적으로 사실에 기반한 주장보다 감정적으로 공감 가는 내용이 더 설득력 있다고 느낀다.,16 Personalities,본능 자아,"[감정반응, 타인인식]",positive
3,4,생활 공간과 업무 공간이 깨끗하게 정돈되어 있다.,16 Personalities,목표 자아,[기타],positive
4,5,압박감이 심한 환경에서도 평정심을 유지하는 편이다.,16 Personalities,목표 자아,[기타],positive
5,6,모르는 사람과 관계를 맺거나 남에게 나를 알리는 것은 매우 부담스럽다.,16 Personalities,목표 자아,[기타],positive
6,7,"효과적으로 작업을 계획하고 우선순위를 정하며, 마감 기한보다 훨씬 일찍 작업을 완료...",16 Personalities,목표 자아,[자기관리],positive
7,8,숫자나 데이터보다는 사람들의 이야기와 감정이 마음에 더 와닿는다.,16 Personalities,본능 자아,"[감정반응, 사회적연결]",positive
8,9,일정이나 목록 등 일을 체계화할 수 있는 도구를 사용하는 것을 좋아한다.,16 Personalities,본능 자아,"[감정반응, 자기관리]",positive
9,10,작은 실수로도 나의 능력이나 지식을 의심할 때가 있다.,16 Personalities,목표 자아,[기타],positive


In [15]:
# 🤔 애매한 문항만 검수 (수동 보정)
ambiguous = df[df['ego']=='목표 자아'][df['themes'].str.contains('기타', na=False)]
print("🔍 수동 검수 필요 문항 (기타 태그):")
print(ambiguous[['No.', '문항', 'themes']].head(10))

# 수동 보정 예시 (필요시 추가)
manual_fix = {
    12: {'ego': '목표 자아', 'themes': ['창의'], 'polarity': 'negative'},
    22: {'ego': '본능 자아', 'themes': ['상상력'], 'polarity': 'negative'}
}

for no, fix in manual_fix.items():
    idx = df[df['No.']==no].index[0]
    df.loc[idx, 'ego'] = fix['ego']
    df.loc[idx, 'themes'] = str(fix['themes'])
    df.loc[idx, 'polarity'] = fix['polarity']

print("✅ 수동 보정 완료!")
print(df['ego'].value_counts())

🔍 수동 검수 필요 문항 (기타 태그):
Empty DataFrame
Columns: [No., 문항, themes]
Index: []
✅ 수동 보정 완료!
ego
목표 자아    474
관계 자아    199
본능 자아    155
Name: count, dtype: int64


/tmp/ipython-input-4289602883.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ambiguous = df[df['ego']=='목표 자아'][df['themes'].str.contains('기타', na=False)]


In [17]:
# 최종 결과 저장
final_result = df[['No.', '문항', '구분', 'ego', 'themes', 'polarity']].copy()
final_result.to_csv('FINAL_question_metadata.csv', index=False, encoding='utf-8')

# 다운로드
files.download('FINAL_question_metadata.csv')
print("🎉 최종 결과 다운로드 완료!")
print("\n📋 다음 단계: 응답 데이터 분석 코드 필요하면 말해줘!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 최종 결과 다운로드 완료!

📋 다음 단계: 응답 데이터 분석 코드 필요하면 말해줘!
